In [1]:
# run this to shorten the data import from the files
import os
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [4]:
# exercise 01

"""
Bootstrap confidence intervals

You previously saw that there is some degree of correlation between the S&P 500 and Bitcoin. One way to measure this would be to look at the correlation coefficient Pearson's R between the two. However, doing so results in only a point estimate. Presumably, at some points in time the correlation between the two is quite close, while at other times they behave very differently. How can you characterize the variability? One approach is to create a bootstrap confidence interval for the correlation coefficient between the two. That's precisely what you'll do now!

A DataFrame of S&P 500 and Bitcoin prices (btc_sp_df) has been loaded for you, as have the packages pandas as pd, NumPy as np, and stats from SciPy.
"""

# Instructions

"""

    Compute the daily percent change of BTC and SP500; use the console to see the columns needed.
    Write a function which computes Pearson's R and only returns R (not the p-value).
    Form a bootstrap confidence interval using this function.
---
Question

Which of the following statements are supported by the results shown in the console?
[95% of the time, the correlation between the daily change in the two assets is between about 18% and 34%.]
"""

# solution

# Compute the daily percent change of each asset
btc_sp_df['Pct_Daily_Change_BTC'] = (btc_sp_df['Open_BTC'] - btc_sp_df['Close_BTC']) / btc_sp_df['Open_BTC']
btc_sp_df['Pct_Daily_Change_SP500'] = (btc_sp_df['Open_SP500'] - btc_sp_df['Close_SP500']) / btc_sp_df['Open_SP500']

# Write a function which returns the correlation coefficient
def pearson_r(x, y):
    return stats.pearsonr(x,y)[0]
  
# Compute a bootstrap confidence interval
ci = stats.bootstrap((btc_sp_df['Pct_Daily_Change_BTC'], btc_sp_df['Pct_Daily_Change_SP500']), 
                     statistic=pearson_r, 
                     vectorized=False, paired=True, n_resamples=1000, random_state=1)



print(ci.confidence_interval)

#----------------------------------#

# Conclusion

"""
By working with a bootstrap confidence interval you were able to take the correlation coefficient and make inference on its range. You now know not just a single value for the correlation, but rather what _range_ of values you are likely to encounter!
"""

ConfidenceInterval(low=0.1797054193594902, high=0.34392714265260166)


'\nBy working with a bootstrap confidence interval you were able to take the correlation coefficient and make inference on its range. You now know not just a single value for the correlation, but rather what _range_ of values you are likely to encounter!\n'

In [9]:
# exercise 02

"""
Bootstrapping vs. normality

You've seen the results of a bootstrap confidence interval for Pearson's R. But what about common situations like making a confidence interval for a mean? Why would you use a bootstrap confidence interval over a "normal" confidence interval coming from stats.norm?

A DataFrame showing investments from venture capital firms (investments_df) has been loaded for you, as have the packages pandas as pd, NumPy as np, and stats from SciPy.
"""

# Instructions

"""

    Select only the companies in the Analytics market.
    Construct a 95% confidence interval for the mean private_equity using the confidence interval function from stats.norm.
    Do the same calculation on private_equity, but using a bootstrap confidence interval instead.
---
Question

These two return quite different results, even though they're estimating the same thing! The primary reason for the difference is that the mean is relatively small and spread out, so the standard error is large. That causes the normal confidence interval to contain negative values.

Why are negative values in the confidence interval problematic for inference?
[Since the average private equity funding cannot be negative, conclusions from this confidence interval are questionable.]

---
Question

While the SciPy function stats.norm.interval() will gladly return a confidence interval for you, that doesn't mean inference drawn from it is valid! The primary issue is that the data is heavily right-skewed (non-normal), with the vast majority of companies (96.8%!) having no private equity funding.

Why, on the other hand, does the bootstrap confidence interval not contain negative values?
[A bootstrap confidence interval is created by sampling from the original data, which does not contain negative values.]
"""

# solution

# Select just the companies in the Analytics market
analytics_df = investments_df[investments_df['market'] == 'Analytics']

# Confidence interval using the stats.norm function
norm_ci = stats.norm.interval(confidence=0.95,
                             loc=analytics_df['private_equity'].mean(),
                             scale=analytics_df['private_equity'].std() / np.sqrt(analytics_df.shape[0]))

# Construct a bootstrapped confidence interval
bootstrap_ci = stats.bootstrap(data=(analytics_df['private_equity'], ),
                              statistic=np.mean)

print('Normal CI:', norm_ci)
print('Bootstrap CI:', bootstrap_ci.confidence_interval)

#----------------------------------#

# Conclusion

"""
The bootstrap confidence interval is also quite broad, spanning roughly six million dollars! However, this reflects the fact that the private equity funding is highly variable, with one company receiving over six hundred million dollars! And yet, the bootstrap confidence interval avoided negative values and only returned plausible values.
"""

Normal CI: (-695062.1822300977, 4049988.4410101348)
Bootstrap CI: ConfidenceInterval(low=324019.53588332323, high=6894712.068408461)


'\nThe bootstrap confidence interval is also quite broad, spanning roughly six million dollars! However, this reflects the fact that the private equity funding is highly variable, with one company receiving over six hundred million dollars! And yet, the bootstrap confidence interval avoided negative values and only returned plausible values.\n'

In [10]:
# exercise 03

"""
Fisher's method in SciPy

Suppose you read five different papers on the role of a healthy diet on cancer diagnoses, with reported p-values loaded in the list p_values = [0.04, 0.13, 0.001, 0.09, 0.01]. In each case the papers are all assessing the null hypothesis that healthy diet has no relationship on cancer diagnosis, while the alternative is that there is a relationship.

Use these p-values in Fisher's method using SciPy.

The package stats from SciPy have been loaded for you.
"""

# Instructions

"""

    Compute both the test statistic and the p-value for this test.
    Print out the p-value for Fisher's method.
    Print out if is True or False that this p-value is significant at 5%.
---
Question

Which of the following statements is supported by the results shown in the console?
[Based on the p-value in the console, out of all five studies at least one of them should have rejected the null hypothesis.]
"""

# solution
p_values = [0.04, 0.13, 0.001, 0.09, 0.01]
# Compute the combined p-value and the p-value for this test
test_statistic, p_value = stats.combine_pvalues(p_values)

# Print out the p-value for the test
print('Test p-value = ', p_value)

# Print out if the p-value is signifcant at 5%
print(p_value < 0.05)

#----------------------------------#

# Conclusion

"""
Fisher's method can be quite complex, since it's a hypothesis test (giving a p-value) about other hypothesis tests (also giving p-values)! You should be proud of being able to take a complicated idea like this and distill it into a sentence. Notice how Fisher's method doesn't tell you _which_ of the tests should have been rejected, only that at least one is. This may sound similar to ANOVA, where you often conclude that _at least one_ mean is different from the others.
"""

Test p-value =  3.284794364823585e-05
True


"\nFisher's method can be quite complex, since it's a hypothesis test (giving a p-value) about other hypothesis tests (also giving p-values)! You should be proud of being able to take a complicated idea like this and distill it into a sentence. Notice how Fisher's method doesn't tell you _which_ of the tests should have been rejected, only that at least one is. This may sound similar to ANOVA, where you often conclude that _at least one_ mean is different from the others.\n"

In [3]:
# exercise 04

"""
Inference using Fisher's method

Fisher's method returns a p-value telling you if at least one of the null hypotheses should have been rejected. The key words here are "at least one".

Consider an example where you are testing the effectiveness of a certain diet on weight loss. This diet has been studied by multiple researchers in different settings, but all are testing the null hypothesis that the diet does not induce weight loss. All researchers report their p-values, and you use them in Fisher's method. Why would it make sense to reject some, but not necessarily all, of the null hypotheses?
"""

# Instructions

"""
Possible Answers
Select one answer

    Some of the researchers might have tested different null hypotheses.


    The samples were different for each researcher, and so not of all of their conclusions are valid.


    The alternative hypotheses may have been different for each researcher, so that determines whether or not each null should be rejected.


    The samples were different for each researcher, and the diet might have worked better for certain samples than others.(Answer)


"""

# solution



#----------------------------------#

# Conclusion

"""
Another way to say this is that there may have been different effect sizes for each sample. This is a natural part of experimentation, and may yield slightly different results for each group. So while one group may have responded very well to the treatment, another group, through no fault of the researcher, may not have.
"""

'\n\n'

### Summarizing Fisher's method

Fisher's method has many moving parts, and it can be tricky to keep them all in your head. In this exercise you'll assess your understanding of what Fisher's method does, and does not, tell you.

Imagine that you read five different papers on the same topic and took the p-value from each paper. Classify each of the statements below as being true, or not necessarily true.

![Solution](/home/nero/Documents/Estudos/DataCamp/Python/courses/Foundations_of_Inference_in_Python/ch04.png)

**Fisher's method can be confusing since it gives a p-value from p-values, each from their own tests! That's quite a brain bender! Great work seeing the connection between Fisher's method and the individual p-values it is assessing.**

In [12]:
# exercise 05

"""
Permutation tests for correlations

How does the volatility of Bitcoin compare to the volatility of the S&P 500?

You previously computed volatility as the percent daily change, which has been stored for you in the Pct_Daily_Change_BTC and Pct_Daily_Change_SP500 columns in your data. The question you want to answer is the extent to which these two values correlate. One way to answer this is through a permutation test. By randomly shuffling values between the S&P 500 and BTC you are able to see what a random outcome would like like, and then compare this to the observed values.

A DataFrame of S&P 500 and Bitcoin prices (btc_sp_df) has been loaded for you, as have the packages pandas as pd, NumPy as np, and stats from SciPy.
"""

# Instructions

"""

    Define a statistic() function which returns just the Pearson R value between two vectors.
    Set your data equal to a tuple containing the volatility of BTC and SP500.
    Conduct a permutation test with this data, statistic, 1000 resamples, and with an alternative hypothesis of greater volatility with Bitcoin.
    Print if the p-value is significant at 5%.

"""

# solution

# Define a function which returns the Pearson R value
def statistic(x, y):
	return stats.pearsonr(x,y)[0]

# Define the data as the percent daily change from each asset
data = (btc_sp_df['Pct_Daily_Change_BTC'],btc_sp_df['Pct_Daily_Change_SP500'])

# Compute a permutation test for the percent daily change of each asset
res = stats.permutation_test(data, statistic, 
           n_resamples=1000,
           vectorized=False, 
           alternative='greater')

# Print if the p-value is significant at 5%
print(res.pvalue < 0.05)

#----------------------------------#

# Conclusion

"""
Notice how the ability to conduct this test hinged only on your ability to write a statistic function and collect data! Hopefully this shows you the power of a permutation test, and how it can be used in a broad range of situations. You put absolutely no assumptions on your data, and yet you were still able to conclude that the observed greater volatility in Bitcoin over SP500 is indeed statistically significant.
"""

True


'\nNotice how the ability to conduct this test hinged only on your ability to write a statistic function and collect data! Hopefully this shows you the power of a permutation test, and how it can be used in a broad range of situations. You put absolutely no assumptions on your data, and yet you were still able to conclude that the observed greater volatility in Bitcoin over SP500 is indeed statistically significant.\n'

### Permutation tests and bootstrapping

When should you use a permutation test, and when should you use a bootstrapped confidence interval? Each of the cards below give a situation, and you are asked to classify them into the best technique to use for the situation described.

![Solution](/home/nero/Documents/Estudos/DataCamp/Python/courses/Foundations_of_Inference_in_Python/ch04_02.png)

**While permutation tests and bootstrapped confidence intervals share many similarities, their usages are still quite different. Being able to clearly delineate between the two allows you to make proper inference by selecting the right tool for the job.**

In [14]:
analytics_df = investments_df[investments_df['market'] == "Analytics"]
non_analytics_df = investments_df[investments_df['market'] != "Analytics"]

In [15]:
# exercise 06

"""
Analyzing skewed data with a permutation test

Permutation tests can be useful for situations which don't satisfy the conditions of the hypothesis tests you already know. In this exercise you'll code up a permutation test using the statsmodels package.

You're interested in comparing the average number of funding rounds between between companies in the analytics space and all other venture-funded companies. While you may be tempted to use a t-test, you can be sure that the number of funding rounds is not normally distributed. Instead, the majority of companies have only one round, with the number of companies with two or more rounds quickly dropping off.

The following have been loaded for you:

    analytics_df - Data on all analytics companies
    non_analytics_df - Data on all other non-analytics companies

"""

# Instructions

"""

    Define a statistic function which, given two samples fundings_group_1 and fundings_group_2, returns the difference in mean number of funding_rounds.
    Conduct a permutation test using the funding_rounds column from each data set, the statistic function you defined, and 100 resamples.
    Print out the resulting p-value of your permutation test.

"""

# solution

# Write a "statistic" function which calculates the difference in means
def statistic(funding_group_1, funding_group_2):
  return (np.mean(funding_group_1) - np.mean(funding_group_2))

# Conduct a permutation test using 100 resamples
perm_result = stats.permutation_test((analytics_df['funding_rounds'], non_analytics_df['funding_rounds']),
                                    statistic=statistic,
                                    n_resamples=100,
                                    vectorized=False)

# Print the p-value
print(perm_result.pvalue)

#----------------------------------#

# Conclusion

"""
Fantastic job executing a permutation test, as well as making it to the end of the course! You now see how, as long as you can define a statistic, you can test pretty much anything, regardless of the underlying distribution! This allows you lots of flexibility when you encounter situations that don't easily fit into the hypothesis testing framework you may have seen before.
"""

0.019801980198019802


"\nFantastic job executing a permutation test, as well as making it to the end of the course! You now see how, as long as you can define a statistic, you can test pretty much anything, regardless of the underlying distribution! This allows you lots of flexibility when you encounter situations that don't easily fit into the hypothesis testing framework you may have seen before.\n"